## Part 1: Preprocessing

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [54]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [56]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [57]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [58]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [59]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

,count
OverTime,
No,780
Yes,322


In [60]:
# Create LabelEncoder instance
label_encoder = LabelEncoder()

# Fit & transform OverTime column
X_train['OverTime'] = label_encoder.fit_transform(X_train['OverTime'])
X_test['OverTime'] = label_encoder.transform(X_test['OverTime'])

# View dtypes
X_train.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [61]:
# Create a StandardScaler
std_scaler = StandardScaler()

# Fit the StandardScaler to the training data
std_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)


In [62]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_encoder.transform(y_train[['Department']])
y_test_department = department_encoder.transform(y_test[['Department']])
y_train_department


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [63]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition = attrition_encoder.transform(y_test[['Attrition']])
y_train_attrition


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [64]:
y_train_department.shape[1]

3

## Create, Compile, and Train the Model

In [127]:
# Find the number of columns in the X training data
number_input_features = len(X_train.columns)
print(number_input_features)

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(units=128, activation='relu', name='shared_layer2')(shared_layer1)

10


In [128]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(units=y_train_department.shape[1], activation='softmax', name='department_output')(department_hidden)


In [129]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=y_train_attrition.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)


In [130]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer1 (Dense)     │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer2 (Dense)     │ (None, 128)            │          8,320 │ shared_layer1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [131]:
# Train the model
model.fit(X_train_scaled,
          [y_train_department, y_train_attrition],
          epochs=100,
          batch_size=32,
          validation_split = 0.2,
          validation_data=(X_test_scaled, [y_test_department, y_test_attrition])
          )


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - attrition_output_accuracy: 0.7713 - attrition_output_loss: 0.5338 - department_output_accuracy: 0.5049 - department_output_loss: 0.6203 - loss: 1.1545 - val_attrition_output_accuracy: 0.8696 - val_attrition_output_loss: 0.3766 - val_department_output_accuracy: 0.6522 - val_department_output_loss: 0.4973 - val_loss: 0.8661
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - attrition_output_accuracy: 0.8266 - attrition_output_loss: 0.4090 - department_output_accuracy: 0.6277 - department_output_loss: 0.4981 - loss: 0.9072 - val_attrition_output_accuracy: 0.8614 - val_attrition_output_loss: 0.3610 - val_department_output_accuracy: 0.6522 - val_department_output_loss: 0.4975 - val_loss: 0.8522
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8612 - attrition_output_loss: 0.3728 - department_output_accuracy: 0.6412 - department_output_loss: 0.4837 - loss: 0.8565 - val_attrition_output_accuracy: 0.8587 -

In [133]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])
results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8246 - attrition_output_loss: 1.8042 - department_output_accuracy: 0.5467 - department_output_loss: 1.2646 - loss: 3.0728 


[2.819342851638794,
 1.2517216205596924,
 1.5420883893966675,
 0.8369565010070801,
 0.54347825050354]

In [136]:
# Print the accuracy for both department and attrition
print("Department prediction accuracy: ", results[4])
print("Attrition prediction accuracy: ", results[3])

Department prediction accuracy:  0.54347825050354
Attrition prediction accuracy:  0.8369565010070801


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.